### Worker register

The following script allows you to get the list of all the workers in the given jobcard. The actual data is inside modules -> wageseekers -> Jobcard holders information -> select the village -> click on the jobcard

In [ ]:
import pandas as pd

In [ ]:
import requests

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
village_list = ['Kandrum','Sagara','Panasavalasa']

In [ ]:
panchayat_list = ['Kandrum','Sagara','Sovva']

In [ ]:
mandal_jc_register = pd.read_csv('/home/ranu/repos/libtechIndiaCrawler/Data/ApJobcardRegister/Dumbriguda_ap_jc_register.csv')

In [ ]:
mandal_jc_register = mandal_jc_register[mandal_jc_register.village_name.isin(village_list)]

In [ ]:
mandal_jc_register = mandal_jc_register[mandal_jc_register.panchayat_name.isin(panchayat_list)]

In [ ]:
mandal_jc_register.panchayat_name.unique()

In [ ]:
mandal_jc_register.block_name.unique()

In [ ]:
mandal_jc_register.village_name.unique()

In [ ]:
mandal_jc_register['Jobcard ID'] = '0' + mandal_jc_register['Jobcard ID'].astype(str)

In [ ]:
mandal_jc_register.village_code = mandal_jc_register.village_code.astype(str).str.rjust(3, "0")

In [ ]:
mandal_jc_register.village_code = mandal_jc_register.panchayat_code.astype(str) + mandal_jc_register.village_code.astype(str)

In [ ]:
headOfTheFamily_dict = dict(zip(mandal_jc_register['Jobcard ID'],mandal_jc_register['Head of Family']))

In [ ]:
village_dict = dict(zip(mandal_jc_register['Jobcard ID'], mandal_jc_register.village_name))

In [ ]:
block_dict = dict(zip(mandal_jc_register['Jobcard ID'],mandal_jc_register.block_name))

In [ ]:
gp_dict = dict(zip(mandal_jc_register['Jobcard ID'],mandal_jc_register.panchayat_name))

In [ ]:
nic_jc_dict = dict(zip(mandal_jc_register['Jobcard ID'],mandal_jc_register['Govt of India JobCard ID']))

In [ ]:
def fetch_workers_register(jobcard):
    
    cookies = {
    'JSESSIONID': '8252084C0821D68FAE4E1B2F7A2768DC.mgnregsapps1',
}


    headers = {
        'Connection': 'keep-alive',
        'Cache-Control': 'max-age=0',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Mobile Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'Referer': 'http://www.nrega.ap.gov.in/Nregs/FrontServlet?requestType=WageSeekersRH&actionVal=JobCardHolder&param=JCHI&type=-1&Ajaxid=go',
        'Accept-Language': 'en-US,en;q=0.9',
    }

    params = (
        ('requestType', 'WageSeekersRH'),
        ('actionVal', 'JobCardHolder'),
        ('hhid', jobcard),
        ('Ajaxid', 'go'),
        ('type', '-1'),
        ('param', 'JCHI'),
    )

    response = requests.get('http://www.nrega.ap.gov.in/Nregs/FrontServlet', headers=headers, params=params, cookies=cookies, verify=False)
    
    cols_list = ['sno','worker_code','name','age','gender','relationship','paying_agency','branch','shg_id','disabled','mphssid']
    
    df = pd.read_html(response.content)[-1]
    
    df.columns = cols_list
    
    return df

In [ ]:
dfs = []
try:

    for rnum,i in enumerate(mandal_jc_register['Jobcard ID']):
        df = fetch_workers_register(i)
        block_name = block_dict[i]
        gp_name = gp_dict[i]
        village_name = village_dict[i]
        nic_jc = nic_jc_dict[i]
        report_date = pd.datetime.now().strftime("%Y-%m-%d")
        df['jobcard_no'] = i
        df['nic_jobcard_no'] = nic_jc
        df['block_name'] = block_name
        df['gp_name'] = gp_name
        df['village_name'] = village_name
        df['report_date'] = report_date
        df['head_of_family'] = headOfTheFamily_dict[i]
        dfs.append(df)

        print(f'{rnum} Workers data fetched for {i} - {village_name}')

except:
    pass
    

In [ ]:
df = pd.concat(dfs)

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df = df.iloc[:,1:]

In [ ]:
df.columns

In [ ]:
cols_list = ['worker_code','name','age','gender','relationship','paying_agency','branch','shg_id','disabled','mphssid']


In [ ]:
cols_list.insert(0,'block_name')

In [ ]:
cols_list.insert(1,'gp_name')

In [ ]:
cols_list.insert(2,'village_name')

In [ ]:
cols_list.insert(3,'jobcard_no')

In [ ]:
cols_list.insert(4,'nic_jobcard_no')

In [ ]:
cols_list.insert(5,'head_of_family')

In [ ]:
df = df[cols_list]

In [ ]:
df.jobcard_no = '~' + df.jobcard_no.astype(str)

In [ ]:
df.to_csv('/home/ranu/repos/libtechIndiaCrawler/Data/workersRegisterAP/nature_dumbriguda.csv')